In [2]:
%pip install nltk -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
 
nltk.download('stopwords')
stopwords = stopwords.words("english")

from xml.etree import ElementTree as ET

[nltk_data] Downloading package stopwords to /home/keddie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import spacy
path_data = "Data/"
!python -m spacy download en_core_web_sm
lemmatizer = spacy.load("en_core_web_sm")

!bash downloads.bash $path_data $model_path

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 11.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
The data were already unpacked


Special case: if the label does not contain "P" at the start (due to possible errors - or just missing in the file)

In [4]:
import re
import string

def get_text_from_xml(xml_file: str, data_file: str) -> pd.DataFrame:
    parsed_dir = ET.parse(path_data + xml_file)
    parsed_dir = parsed_dir.getroot()

    data = {"ID": [], "Text": []}
    for child_node in parsed_dir:
        id = child_node.attrib["id"]
        if "P" != id[0] and id[0] != "A":
            id = "P" + id
        data["ID"].append(id)
        data["Text"].append(child_node.text)

    data = pd.DataFrame(data)
    data["Text"] = data["Text"].apply(data_preprocessing)
    data.to_pickle(data_file)

    return data

# main processing: keep compound nouns, lemmatize, stop word removal, remove numbers
def data_preprocessing(data: str) -> list:
    data = data.lower()
    # @TODO try without compound nouns
    # !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ from string.punctuation
    # keep compund nouns
    data = regexp_tokenize(data, r"[\w]+(?:[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~][\w]+[()*[\]{}]?){1,}|[\w]+|(?:[([{]\w+[)\]}])+")

    ### @TODO: 
    # * lemmatize
    aux_data = data
    data = " ".join(data)
    lemmatized_data = lemmatizer(data)

    aux = []
    for aux_token in aux_data:
        found = False
        for token in lemmatized_data:
            if token.text == aux_token:
                aux.append(token.lemma_)
                found = True

            if found:
                break

        if not found:
            aux.append(aux_token)

    data = aux

    # stop word removal
    data = [word for word in data if word not in stopwords]


    #### remove numbers
    # data = [word for word in data if not re.search(r"[0-9]+", word)]

    return data

directive_data = get_text_from_xml("DIR_EN_32002L0044.xml", "directive_data.pickle")
provision_data = get_text_from_xml("NIM_EN.xml", "provision_data.pickle")

In [72]:
directive_data.head()

,ID,Text
0,A1_1,"[directive, individual, directive, within, mea..."
1,A1_2,"[requirement, directive, shall, apply, activit..."
2,A1_3,"[directive, shall, apply, fully, whole, area, ..."
3,A2,"[purpose, directive, follow, term, shall, mean..."
4,A3_1,"[exposure, limit, value, action, value, hand-a..."


In [5]:
for text in directive_data["Text"]:
    print(text)
    print()

['directive', '16th', 'individual', 'directive', 'within', 'meaning', 'article', '16(1)', 'directive', '89/391/eec', 'lay', 'minimum', 'requirement', 'protection', 'worker', 'risk', 'health', 'safety', 'arise', 'likely', 'arise', 'exposure', 'mechanical', 'vibration']

['requirement', 'directive', 'shall', 'apply', 'activity', 'worker', 'likely', 'expose', 'risk', 'mechanical', 'vibration', 'work']

['directive', '89/391/eec', 'shall', 'apply', 'fully', 'whole', 'area', 'refer', 'paragraph', '1', 'without', 'prejudice', 'stringent', 'and/or', 'specific', 'provision', 'contain', 'directive']

['purpose', 'directive', 'follow', 'term', 'shall', 'mean', 'hand-arm', 'vibration', 'mechanical', 'vibration', 'transmit', 'human', 'hand-arm', 'system', 'entail', 'risk', 'health', 'safety', 'worker', 'particular', 'vascular', 'bone', 'joint', 'neurological', 'muscular', 'disorder', 'whole-body', 'vibration', 'mechanical', 'vibration', 'transmit', 'whole', 'body', 'entail', 'risk', 'health', 'saf